## Submission Notebook for TEAM 17

<b> Install your packages below: </b>

In [ ]:
!pip install -r requirements.txt

In the section below, you must run your methodology for solving the problem from start to finish :

In [ ]:
import pickle

with open('building_1.pkl', 'rb') as f:
    building_1 = pickle.load(f)

with open('building_2.pkl', 'rb') as f:
    building_2 = pickle.load(f)
    
with open('building_3.pkl', 'rb') as f:
    building_3 = pickle.load(f)

buildings = [building_1, building_2, building_3]

In [ ]:
import time # Necessary to evaluate frugality
from pymgrid.Environments.pymgrid_cspla import MicroGridEnv # Imposed Environment
import numpy as np

## Import your favourite Deep Learning library for RL and other packages here

In [ ]:
"""
Below is an environment initialization without a Deep RL library, the code can vary depending on which library you 
use
"""
building_environments = [MicroGridEnv(env_config={'microgrid':buildings[i]}) for i in range(3)]


<b> 3) Training of the agent </b>

In [ ]:
train_start = time.process_time()

"""
Train code
"""

train_end = time.process_time()
train_frugality = train_end - train_start

<b> 4) Test of the agent </b>

In [ ]:
"""
Below is an example for a Random Agent 

Note :
* To make your work as reproductible as possible, have a full-greedy approach (no exploration) on the test buildings
* If your algorithm has some unavoidable randomness, consider running it for many loops and return a
  mean profitability and mean frugality
  
"""

test_start = time.process_time()


test_start = time.process_time()
total_cost = [0,0,0]

for i,building_env in enumerate(building_environments):
    
    obs = building_env.reset(testing=True)
    done = False
    while not done:
        action = np.random.randint(building_env.action_space.n)
        obs, reward, done, info = building_env.step(action)
        total_cost[i]+=reward

test_end = time.process_time()

test_frugality = test_end - test_start

In [ ]:
frugality = train_frugality + test_frugality

<b> 5) Store & Export Results in JSON format </b>

In [ ]:
final_results = {
    "building_1_performance" : total_cost[0],
    "building_2_performance" : total_cost[1],
    "building_3_performance" : total_cost[2],
    "frugality" : train_frugality + test_frugality,
}
print(final_results)

In [ ]:
team_name = "team17"

with open(team_name + '.txt', 'w') as json_file:
    json.dump(final_results, json_file)